In [ ]:
pip install numpy scikit-learn seaborn matplotlib statsmodels

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

IMPORT DATA SET

In [ ]:
# Cargar el conjunto de datos
url = 'https://raw.githubusercontent.com/d2cml-ai/CausalAI-Course/main/data/wage2015_subsample_inference.csv'

In [ ]:
df = pd.read_csv(url)
print(df.head())

In [ ]:
# Asegurarse que los nombres de las variables están bien cargados
print(df.columns)

sex, 1 equals Female
shs: "some highschool"
hsg: "highschool graduate"
scl: "some college"
clg: "college graduate"
ad: "advanced degree"
mw: "midwest"
so: "south"
we: "west"
ne: "northeast"
occ2: "occupation code"
ind2: "industry code"

MISSING VALUES

In [ ]:
# Revisar los missing values
missing_values = df.isnull().sum()
print("Valores faltantes:", missing_values)

In [ ]:
# Estadísticas descriptivas
descriptive_stats = df.describe(percentiles=[0.25, 0.5, 0.75])
print("Estadísticas descriptivas:", descriptive_stats)

In [ ]:
#WAGE: La media del salario es 23.41, con una desviación estándar de 21.08, lo que indica que hay una gran variabilidad en los salarios. 
# El salario mediano es 19.23, lo que significa que el 50% de los sujetos ganan menos de esta cantidad, 
# lo que indica una distribución sesgada positivamente, ya que la media es mayor que la mediana. 
# El salario máximo es bastante alto en comparación con la mayoría de los salarios, lo que sugiere la presencia de valores atípicos.

In [ ]:
#lwage: se usa para normalizar la distribución de salarios. 
#Los percentiles son similares a los del salario, pero en una escala logarítmica, lo que ayuda a reducir el impacto de valores extremos

In [ ]:
#sex: Con una media de 0.444, aproximadamente el 44.4% de los individuos en la muestra son mujeres (si 1 = mujer). 
# La desviación estándar cercana a 0.5 es típica para una variable binaria con proporciones relativamente equilibradas

In [ ]:
#shs: solo el 2.3% de la muestra ha completado "algo de secundaria"
#hsg:  el 24.4% están graduados de la escuela secundaria
#scl: 27.8% de la muestra ha completado "algo de la universidad"
#clg: 31.8% son graduados universitarios
#ad: 13.7% tienen un titulo avanzado
#mw: 25.9% pertenen al medio oeste
#we: 21.6% son parte del sur
#ne: 22.7% del norte

In [ ]:
#mujeres con título universitario o superior tienen un salario correspondiente al 25% más rico de la muestra

# Filtrar las mujeres con título universitario o superior
mujeres_tituladas_superior_df = df[(df['sex'] == 1) & ((df['clg'] == 1) | (df['ad'] == 1))]

# Determinar el umbral del 25% más rico para toda la muestra
salariado_75 = df['wage'].quantile(0.75)

# Filtrar las mujeres con salario en el 25% más alto
mujeres_mas_asalariadas_df = mujeres_tituladas_superior_df[mujeres_tituladas_superior_df['wage'] >= salariado_75]


print(mujeres_mas_asalariadas_df)

In [ ]:
# Contar cuántas mujeres cumplen con ambas condiciones
contar_mujeres = mujeres_mas_asalariadas_df.shape[0]

print(contar_mujeres)

In [ ]:
#hombres con título de bachillerato o inferior tienen un salario correspondiente al 25 % más rico de la muestra

#hombres con bachillerato o inferor
hombres_bachillerato_inferior_df =df[(df['sex'] == 0) & ((df['hsg'] == 1)|(df["shs"]==1))]

# Filtrar los hombres con salario en el 25% más alto
hombres_mas_asalariados_df = hombres_bachillerato_inferior_df[hombres_bachillerato_inferior_df['wage'] >= salariado_75]

print(hombres_mas_asalariados_df)

In [ ]:
# Contar cuántos hombres cumplen con ambas condiciones
contar_hombres = hombres_mas_asalariados_df.shape[0]

print(contar_hombres)


In [ ]:
# Crear un marco de datos con solo la variable 'wage'
lwage_df = df[['lwage']]

# Crear un marco de datos con todas las variables excepto 'wage'
datos_df = df.drop(columns=['wage','lwage'])

In [ ]:
print(lwage_df)
print(datos_df)

DATA WRANGLING

Modelo Básico

In [ ]:
# Crear variables para regresión
# a) Variable dependiente
Y = df['lwage']

In [ ]:
# Predictores modelo básico
X_basic = df[['sex', 'exp1', 'hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne']]

In [ ]:
# Convertir occ2 e ind2 a variables dummy
X_basic = pd.concat([X_basic, pd.get_dummies(df['occ2'], drop_first=True), pd.get_dummies(df['ind2'], drop_first=True)], axis=1)

Modelo Flexible 

In [ ]:
# Modelo flexible: añadir polinomios e interacciones de experiencia
df['exp2'] = df['exp1'] ** 2
df['exp3'] = df['exp1'] ** 3
df['exp4'] = df['exp1'] ** 4

In [ ]:
X_flexible = X_basic.copy()
X_flexible['exp2'] = df['exp2']
X_flexible['exp3'] = df['exp3']
X_flexible['exp4'] = df['exp4']

In [ ]:
# Crear interacciones
interaction_vars = ['hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne']
for exp_var in ['exp1', 'exp2', 'exp3', 'exp4']:
    for var in interaction_vars:
        X_flexible[f'{exp_var}*{var}'] = df[exp_var] * df[var]

Modelo Extraflexible

In [ ]:
# Modelo extraflexible: todas las interacciones bidireccionales
X_extraflexible = X_flexible.copy()


In [ ]:
import itertools

In [ ]:
# Restablecer los índices para evitar duplicados
X_flexible = X_flexible.reset_index(drop=True)
X_extraflexible = X_extraflexible.reset_index(drop=True)

# Crear combinaciones de pares de columnas sin repeticiones
for var1, var2 in itertools.combinations(X_flexible.columns, 2):
    col_name = f'{var1}*{var2}'
    if col_name not in X_extraflexible.columns:
        # Verificar si las columnas son multidimensionales y seleccionar la primera dimensión si es el caso
        if len(X_flexible[var1].shape) > 1:
            column1 = X_flexible[var1].iloc[:, 0]  # Seleccionar la primera subcolumna
        else:
            column1 = X_flexible[var1]

        if len(X_flexible[var2].shape) > 1:
            column2 = X_flexible[var2].iloc[:, 0]  # Seleccionar la primera subcolumna
        else:
            column2 = X_flexible[var2]

        # Multiplicar columnas de la misma dimensión
        X_extraflexible[col_name] = column1 * column2


In [ ]:
# Dividir en entrenamiento y prueba
X_train_basic, X_test_basic, Y_train, Y_test = train_test_split(X_basic, Y, test_size=0.2, random_state=42)
X_train_flexible, X_test_flexible, _, _ = train_test_split(X_flexible, Y, test_size=0.2, random_state=42)
X_train_extraflexible, X_test_extraflexible, _, _ = train_test_split(X_extraflexible, Y, test_size=0.2, random_state=42)

In [ ]:
# Ajustar modelos y calcular métricas
def fit_and_evaluate(X_train, X_test, Y_train, Y_test):
    model = LinearRegression()
    model.fit(X_train, Y_train)
    
    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)
    
    mse_train = mean_squared_error(Y_train, Y_pred_train)
    mse_test = mean_squared_error(Y_test, Y_pred_test)
    
    r2_train = r2_score(Y_train, Y_pred_train)
    r2_test = r2_score(Y_test, Y_pred_test)
    
    return mse_train, mse_test, r2_train, r2_test


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression


In [ ]:
# Convertir los nombres de las columnas a cadenas en X_train_basic y X_test_basic
X_train_basic.columns = X_train_basic.columns.astype(str)
X_test_basic.columns = X_test_basic.columns.astype(str)

# Llamada a la función fit_and_evaluate para el modelo básico
mse_train_basic, mse_test_basic, r2_train_basic, r2_test_basic = fit_and_evaluate(X_train_basic, X_test_basic, Y_train, Y_test)

In [ ]:
# Convertir los nombres de las columnas a cadenas en X_train_flexible y X_test_flexible (si es necesario)
X_train_flexible.columns = X_train_flexible.columns.astype(str)
X_test_flexible.columns = X_test_flexible.columns.astype(str)

# Llamada a la función fit_and_evaluate para el modelo flexible
mse_train_flexible, mse_test_flexible, r2_train_flexible, r2_test_flexible = fit_and_evaluate(X_train_flexible, X_test_flexible, Y_train, Y_test)

In [ ]:
# Convertir los nombres de las columnas a cadenas en X_train_flexible y X_test_flexible (si es necesario)
X_train_extraflexible.columns = X_train_extraflexible.columns.astype(str)
X_test_extraflexible.columns = X_test_extraflexible.columns.astype(str)

# Llamada a la función fit_and_evaluate para el modelo flexible
mse_train_extraflexible, mse_test_extraflexible, r2_train_extraflexible, r2_test_extraflexible = fit_and_evaluate(X_train_extraflexible, X_test_extraflexible, Y_train, Y_test)

In [ ]:
# Imprimir resultados de los tres modelos
print(f"Modelo Básico - MSE Train: {mse_train_basic}, MSE Test: {mse_test_basic}, R² Train: {r2_train_basic}, R² Test: {r2_test_basic}")
print(f"Modelo Flexible - MSE Train: {mse_train_flexible}, MSE Test: {mse_test_flexible}, R² Train: {r2_train_flexible}, R² Test: {r2_test_flexible}")
print(f"Modelo Extraflexible - MSE Train: {mse_train_extraflexible}, MSE Test: {mse_test_extraflexible}, R² Train: {r2_train_extraflexible}, R² Test: {r2_test_extraflexible}")